## 广播

【我的理解】
n维张量A, B的矩阵乘法，要求前面 n-2 维都相等, 最后2个维度必须是 (m, n), (n, k) 的符合矩阵乘法的要求
 - 【错误点】前面的维度（即 batch 维度）需要满足广播（broadcasting）规则，而不仅仅是“完全相等”。

【理解2】广播规则，则对应的维度必须是整数倍，否则不能广播？
 - 【错误点】广播看的是“相等或为1”，而不是“整数倍”

| 情况                   | 能否广播？ | 原因                     |
| -------------------- | ----- | ---------------------- |
| `(3, 4)` vs `(3, 4)` | ✅     | 相等                     |
| `(1, 4)` vs `(3, 4)` | ✅     | 1 可扩展                  |
| `(2, 6)` vs `(3, 6)` | ❌     | 2≠3 且无 1               |
| `(4, 5)` vs `(2, 5)` | ❌     | 4≠2 且无 1（即使 4 是 2 的倍数） |


【问题】为什么不能？


|你的想法|实际规则|原因|
|---|---|---|
|“(2,5) 重复一次变 (4,5)”|❌ 不允许|广播只允许沿 **size=1** 的维度拉伸|
|广播 = 任意重复|❌|广播 = **无歧义、确定性的虚拟复制**|
|整数倍就能广播|❌|必须满足：**相等 或 其中一个是 1**|

> 💡 **记住**：广播是为了方便“标量/向量/矩阵”与高维张量运算，**不是为了任意形状的自动适配**。当需要非标准对齐时，请显式使用 `repeat`、`expand`、`reshape` 等操作。




## einsum

In [1]:
import torch 
A = torch.randn(2, 3, 4)
B = torch.randn(2, 4, 5)

# 方法1: torch.einsum
C1 = torch.einsum('bij,bjk->bi', A, B)  # 先乘再对 k 求和
print(C1)

# 方法2: einops（需组合）
from einops import einsum
C2 = einsum(A, B, 'b i j, b j k -> b i')  # einops 也支持 einsum!
print(C2)

# 方法3: 原生
C3 = (A @ B).sum(dim=-1)
print(C3)

tensor([[-2.0204, -1.5683, -0.6974],
        [ 1.2406,  0.5915,  0.8927]])
tensor([[-2.0204, -1.5683, -0.6974],
        [ 1.2406,  0.5915,  0.8927]])
tensor([[-2.0204, -1.5683, -0.6974],
        [ 1.2406,  0.5915,  0.8927]])


### 例子1: 批量矩阵乘法

In [5]:
import torch
from einops import rearrange, einsum

batch = 10
seq_len = 20
d_in = 5
d_out = 4
D = torch.randn(batch, seq_len, d_in)
A = torch.randn(d_out, d_in)

Y = D @ A.T

assert Y.shape == (batch, seq_len, d_out)
print(Y.shape)

Y = einsum(D, A, "batch seq_len d_in, d_out d_in -> batch seq_len d_out")
print(Y.shape)

Y = einsum(D, A, "... d_in, d_out d_in -> ... d_out")
print(Y.shape)



torch.Size([10, 20, 4])
torch.Size([10, 20, 4])
torch.Size([10, 20, 4])


### 例子2：广播操作

In [6]:
from einops import rearrange, einsum

images = torch.randn(64, 128, 128, 3) # (batch, height, width, channel)
dim_by = torch.linspace(start=0.0, end=1.0, steps=10)

## Reshape and multiply  
dim_value = rearrange(dim_by, "dim_value -> 1 dim_value 1 1 1")  
images_rearr = rearrange(images, "b height width channel -> b 1 height width channel")  
dimmed_images = images_rearr * dim_value

## Or in one go:
dimmed_images2 = einsum(
    images, dim_by,
    "batch height width channel, dim_value -> batch dim_value height width channel"
)

print(torch.equal(dimmed_images, dimmed_images2))
print(dimmed_images.shape)

True
torch.Size([64, 10, 128, 128, 3])


### 例子3：像素混合

## 索引

PyTorch 支持多种索引方式，主要包括：

1. **基本切片（Basic Slicing）**
    - 使用 `:`、整数、`...` 等。
    - 返回的是原张量的**视图（view）**，不复制数据。
    - 例如：`t[0]`, `t[:, 1:3]`
2. **高级索引（Advanced Indexing）**
    - 使用 **整数列表/数组** 或 **布尔掩码**。
    - 返回的是**新张量（副本）**，会复制数据。
    - 例如：`t[[0, 2]]`, `t[torch.tensor([0,2])]`

> ⚠️ 注意：当你用 `torch.LongTensor([0, 2])` 这样的张量作为索引时，就进入了**高级索引**的范畴。

`indices` 是一个一维 LongTensor，PyTorch 默认将其用于第一个维度（dim=0）。
所以它会取出第 0 行和第 2 行。

📌 **关键点**：`t1[indices]` 实际上是 `t1[indices, :]` 的简写（省略了后面的 `:`）。

注意：

- `t1[indices]` 是 Python 的 `__getitem__` 语法糖，更简洁。
- `index_select` 更**显式、安全、可读性强**，尤其在复杂场景中推荐使用。

In [11]:
t1 = torch.arange(12).reshape((3, 4))
indices = torch.LongTensor([0, 2])

print(t1[indices])
print(t1[:, indices])
print(torch.index_select(t1, 0, indices))
print(torch.index_select(t1, 1, indices))

# 但注意：不能直接写 t1[indices, indices]！
# 因为这会触发“广播式高级索引”，结果是 (2,) 而不是 (2,2)
print(t1[indices, indices])  # 输出:  ← 只取 (0,0) 和 (2,2)
print("=" * 40)


tensor([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11]])
tensor([[ 0,  2],
        [ 4,  6],
        [ 8, 10]])
tensor([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11]])
tensor([[ 0,  2],
        [ 4,  6],
        [ 8, 10]])
tensor([ 0, 10])


In [ ]:
res = t1[[1, 0, 1, 2, 2]]
print(res, res.shape)
res = t1[[1, 0, 1, 2, 2],:]  # 等价表达式, 从第0维 选择多个行
print(res, res.shape)

tensor([[ 4,  5,  6,  7],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11]]) torch.Size([5, 4])
tensor([[ 4,  5,  6,  7],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11]]) torch.Size([5, 4])


In [13]:
print(t1, "t1")
idx2 = torch.LongTensor([[0, 2], [0, 2]])
res2 = t1[idx2]
print(res2, res2.shape)
print(torch.gather(t1, dim=0, index=idx2))
# take_along_dim

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]) t1
tensor([[[ 0,  1,  2,  3],
         [ 8,  9, 10, 11]],

        [[ 0,  1,  2,  3],
         [ 8,  9, 10, 11]]]) torch.Size([2, 2, 4])
tensor([[0, 9],
        [0, 9]])


这是一个非常经典的 **高级索引（Advanced Indexing）** 案例。在 2026 年的 PyTorch 版本中，其逻辑依然遵循 NumPy 的广播规则。

为什么结果是 `[2, 2, 4]`？

当你对一个 2D 张量 `t1` 使用一个 2D 索引张量 `idx2` 时，PyTorch 的处理逻辑如下：

1. **索引层级定位**：你只提供了一个索引张量 `idx2`，因此 PyTorch 将其作用于 `t1` 的**第一个维度（Dim 0，即“行”）** [1]。
2. **形状映射**：
    - `t1` 的形状是 `(3, 4)`。其中 `dim 0` 大小为 3，`dim 1` 大小为 4。
    - `idx2` 的形状是 `(2, 2)`。
3. **元素替换**：
    - PyTorch 会遍历 `idx2` 中的每一个值，将其替换为 `t1` 中对应的“行”。
    - `idx2` 中的元素是 `0` 和 `2`。在 `t1` 中，第 0 行是 `[0, 1, 2, 3]`，第 2 行是 `[8, 9, 10, 11]`。
4. **最终形状计算**：
    - 结果的前两个维度由 **索引张量 `idx2` 的形状** 决定：`(2, 2)`。
    - 结果的剩余维度由 **被索引张量 `t1` 剩余的维度** 决定：`(4,)`。
    - 因此，最终形状为 `(2, 2, 4)`。

内部对应关系拆解：

- `res2[0, 0]` 对应 `t1[idx2[0, 0]]` -> `t1[0]` -> `[0, 1, 2, 3]`
- `res2[0, 1]` 对应 `t1[idx2[0, 1]]` -> `t1[2]` -> `[8, 9, 10, 11]`
- `res2[1, 0]` 对应 `t1[idx2[1, 0]]` -> `t1[0]` -> `[0, 1, 2, 3]`
- `res2[1, 1]` 对应 `t1[idx2[1, 1]]` -> `t1[2]` -> `[8, 9, 10, 11]`

如果你想得到 `(2, 2)` 而不是 `(2, 2, 4)` 怎么办？

如果你希望结果直接定位到具体的标量元素，你需要同时提供 **Dim 1（列）** 的索引。

例如，如果你想在 `t1` 中根据 `idx2` 提取特定的元素，你需要让索引张量的形状匹配：

In [49]:
# 假设我们要提取 (0,0), (2,2) 和 (0,0), (2,2) 位置的四个元素
row_idx = torch.LongTensor([[0, 2], [0, 2]])
col_idx = torch.LongTensor([[0, 2], [0, 2]])

res = t1[row_idx, col_idx] 
print(res, res.shape) # 输出 torch.Size([2, 2])


tensor([[ 0, 10],
        [ 0, 10]]) torch.Size([2, 2])


Multiple Dimensions (Coordinate Indexing): When multiple integer index tensors are provided, they broadcast together to form a set of coordinates. The resulting tensor's shape will be the broadcasted shape of the index tensors.

In [39]:
x = torch.arange(12).reshape(3, 4)
# x:
# tensor([[ 0,  1,  2,  3],
#         [ 4,  5,  6,  7],
#         [ 8,  9, 10, 11]])

rows = torch.tensor([0, 2])
cols = torch.tensor([1, 3])
# Select elements at (0, 1) and (2, 3)
selected = x[rows, cols]
# selected is tensor([1, 11])
print(selected)

tensor([ 1, 11])


Boolean Tensor Indexing

You can use a BoolTensor as a mask to select elements corresponding to True values. 
 - The boolean tensor must have the same shape as the original tensor (or be broadcastable) in the dimensions being masked. The output is a 1D tensor containing all the selected elements.

In [41]:
x = torch.arange(4).reshape(2, 2)
# x:
# tensor([[0, 1],
#         [2, 3]])

mask = torch.tensor([[True, False], [False, True]])
selected = x[mask]
print(selected)

tensor([0, 3])


## 初始化

In [7]:
t1 = torch.ones((2, 3))
torch.nn.init.trunc_normal_(t1, a=-0.5, b=0.5)
print(t1)

tensor([[-0.4119, -0.4434,  0.4310],
        [ 0.3909, -0.2405,  0.0515]])


In [8]:
torch.sqrt(torch.tensor(9.0)).item()

3.0

## nn.Parameter

In [2]:
import torch
import torch.nn as nn

weight1 = torch.randn(2, 3)
print(weight1.requires_grad)

weight2 = nn.Parameter(torch.randn(2, 3))
print(weight2.requires_grad)

False
True


## 练习题：高级索引

In [10]:
import torch
import torch.nn as nn

class NaiveEmbedding(nn.Module):
    """繁琐写法：使用 index_select + reshape"""
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embed_matrix = nn.Parameter(torch.randn(vocab_size, embed_dim))
    
    def forward(self, token_ids):
        # TODO: 实现 index_select 的繁琐写法（4步）
        # 1. 保存原始形状（除最后一维外）
        original_shape = token_ids.shape
        
        # 2. 将 token_ids 展平为 1D
        flat_ids = token_ids.reshape(-1)
        print("flat_ids", flat_ids.shape)
        
        # 3. 使用 index_select 提取向量（注意：index_select 要求索引为1D）
        flat_embeds = torch.index_select(self.embed_matrix, 0, flat_ids)
        
        # 4. 恢复形状：原始形状 + (embed_dim,)
        new_shape = original_shape + (flat_embeds.shape[-1], )
        return flat_embeds.reshape(new_shape) # ← 用户填写


class OptimizedEmbedding(nn.Module):
    """简洁写法：直接使用高级索引"""
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embed_matrix = nn.Parameter(torch.randn(vocab_size, embed_dim))
    
    def forward(self, token_ids):
        # TODO: 用一行代码实现 embedding lookup
        return self.embed_matrix[token_ids]


# ==================== 测试框架（未填写核心逻辑时必然失败） ====================
def test_embedding_consistency():
    vocab_size, embed_dim = 100, 64
    naive_emb = NaiveEmbedding(vocab_size, embed_dim)
    opt_emb = OptimizedEmbedding(vocab_size, embed_dim)
    opt_emb.embed_matrix.data = naive_emb.embed_matrix.data.clone()
    
    # 测试1: 2D 张量 [batch, seq_len]
    token_ids_2d = torch.randint(0, vocab_size, (4, 10))
    
    # 未填写时：naive_emb 返回 None → 触发 TypeError
    naive_out = naive_emb(token_ids_2d)  # ← 必然失败！用户需实现
    opt_out = opt_emb(token_ids_2d)      # ← 必然失败！用户需实现
    
    # 以下断言仅在用户正确实现后通过
    assert naive_out is not None, "❌ NaiveEmbedding 未实现！"
    assert opt_out is not None, "❌ OptimizedEmbedding 未实现！"
    assert torch.allclose(naive_out, opt_out), "❌ 输出不一致！"
    assert opt_out.shape == (4, 10, embed_dim), f"❌ 形状错误: {opt_out.shape}"
    
    # 测试2: 3D 张量验证形状传播
    token_ids_3d = torch.randint(0, vocab_size, (2, 5, 3))
    output = opt_emb(token_ids_3d)
    assert output.shape == (2, 5, 3, embed_dim), f"❌ 3D形状错误: {output.shape}"
    
    # 附加挑战：负数索引（仅 OptimizedEmbedding 支持）
    neg_ids = torch.tensor([[-1, 0], [1, -2]])  # -1 表示最后一个元素
    try:
        naive_emb(neg_ids)  # 应抛出 RuntimeError（index_select 不支持负索引）
        raise AssertionError("❌ NaiveEmbedding 应拒绝负索引！")
    except Exception as e:
        print("异常index token id，抛异常符合行为。", str(e))
        pass  # 预期行为
    
    # OptimizedEmbedding 应正确处理负索引
    neg_out = opt_emb(neg_ids)
    assert neg_out.shape == (2, 2, embed_dim), "❌ 负索引输出形状错误"
    # 验证 -1 确实指向最后一个元素
    assert torch.allclose(neg_out[0,0], opt_emb.embed_matrix[-1]), "❌ 负索引逻辑错误"

if __name__ == "__main__":
    try:
        test_embedding_consistency()
        print("✅ 恭喜！所有测试通过，您已掌握高级索引精髓")
    except Exception as e:
        import traceback
        traceback.print_exc() 
        print(f"❌ 测试失败: {type(e).__name__}: {e}")
        print("\n💡 提示：请完成以下核心实现：")
        print("  1. NaiveEmbedding.forward() 中的 4 步 reshape + index_select")
        print("  2. OptimizedEmbedding.forward() 中的单行高级索引")

flat_ids torch.Size([40])
flat_ids torch.Size([4])
异常index token id，抛异常符合行为。 index out of range in self
✅ 恭喜！所有测试通过，您已掌握高级索引精髓


## keepdim

In [30]:
images = torch.randn(64, 128, 3)
res0 = torch.mean(images)
res1 = torch.mean(images, dim=2)
res2 = torch.mean(images, dim=2, keepdim=True)

print(res0, res0.shape)
print(res1.shape, res2.shape)

tensor(0.0022) torch.Size([])
torch.Size([64, 128]) torch.Size([64, 128, 1])


In [26]:
images - res1

RuntimeError: The size of tensor a (3) must match the size of tensor b (128) at non-singleton dimension 2

In [28]:
print((images - res2).shape)

torch.Size([64, 128, 3])


## Rope相关-range,半旋转

In [40]:
theta = 1000
d_k = 50
invert = theta ** (-torch.arange(0, d_k/2, 1)/(d_k/2))
print(invert.shape)
print(invert[0], invert[-1])
print(1000 ** (-24/25))

irange = -torch.arange(0, d_k/2, 1)/(d_k/2)
print(irange.shape)
print(irange[0], irange[-1])

torch.Size([25])
tensor(1.) tensor(0.0013)
0.0013182567385564075
torch.Size([25])
tensor(-0.) tensor(-0.9600)


In [42]:
torch.cos(3 * invert)

tensor([-0.9900, -0.6480, -0.1549,  0.2583,  0.5458,  0.7293,  0.8410,  0.9074,
         0.9464,  0.9690,  0.9821,  0.9897,  0.9941,  0.9966,  0.9980,  0.9989,
         0.9993,  0.9996,  0.9998,  0.9999,  0.9999,  1.0000,  1.0000,  1.0000,
         1.0000])

In [48]:
seq_len = 10
token_positions = torch.arange(seq_len)
print(token_positions.shape)
print(token_positions)
token_positions = token_positions.unsqueeze(0)
torch.unsqueeze
print(token_positions.shape)
print(token_positions)

torch.Size([10])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
torch.Size([1, 10])
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])


你观察到的这个行为是完全正确的，这是 PyTorch `torch.repeat_interleave()` 函数的一个特殊重载版本的行为。

**解释**

当你调用 `torch.repeat_interleave(arr)` 只传入一个参数时，PyTorch 会调用以下重载：

```python
repeat_interleave(repeats, *, output_size=None) -> Tensor
```

在这种情况下，**传入的张量被解释为 `repeats`（重复次数）参数**，而不是 `input`（输入张量）参数。

根据 PyTorch 官方文档的说明：
> "If the `repeats` is `tensor([n1, n2, n3, ...])`, then the output will be `tensor([0, 0, ..., 1, 1, ..., 2, 2, ..., ...])` where `0` appears `n1` times, `1` appears `n2` times, `2` appears `n3` times, etc." 

**具体分析你的例子**

```python
seq_len = 10
arr = torch.arange(seq_len)  # tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
```

当调用 `torch.repeat_interleave(arr)` 时：
- `arr` 被当作 `repeats` = `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`
- 输出规则：**数字 0 重复 0 次，数字 1 重复 1 次，数字 2 重复 2 次，...，数字 9 重复 9 次**

所以输出是：
```
tensor([1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 
        7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9])
```

验证：
- 0 出现 0 次 → 没有 0
- 1 出现 1 次 → `[1]`
- 2 出现 2 次 → `[2, 2]`
- 3 出现 3 次 → `[3, 3, 3]`
- ...
- 9 出现 9 次 → `[9, 9, 9, 9, 9, 9, 9, 9, 9]`


这个单参数重载版本的设计是为了方便生成特定模式的序列，类似于 `numpy.repeat` 的某些用法。

In [52]:
seq_len = 10
arr = torch.arange(seq_len)
torch.repeat_interleave(arr)

tensor([1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7,
        7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [50]:
torch.repeat_interleave(arr, repeats=2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9])

In [53]:
arr = torch.arange(12).reshape((3, 4))

torch.repeat_interleave(arr, repeats=3, dim=1)

tensor([[ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3],
        [ 4,  4,  4,  5,  5,  5,  6,  6,  6,  7,  7,  7],
        [ 8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11]])

In [58]:
arr = torch.arange(16).reshape((4, 4))
arr.unbind(dim=1)

(tensor([ 0,  4,  8, 12]),
 tensor([ 1,  5,  9, 13]),
 tensor([ 2,  6, 10, 14]),
 tensor([ 3,  7, 11, 15]))

In [59]:
arr.unbind(dim=0)

(tensor([0, 1, 2, 3]),
 tensor([4, 5, 6, 7]),
 tensor([ 8,  9, 10, 11]),
 tensor([12, 13, 14, 15]))

In [62]:
import einops
x1 = torch.arange(3).reshape((1, 3))
x2 = x1 + 3
print(x1)
print(x2)
res = torch.stack((-x2, x1), dim=-1)
print(res)
einops.rearrange(res, "... d j -> ... (d j)")

tensor([[0, 1, 2]])
tensor([[3, 4, 5]])
tensor([[[-3,  0],
         [-4,  1],
         [-5,  2]]])


tensor([[-3,  0, -4,  1, -5,  2]])

In [64]:
import einops
x0 = torch.arange(6)
print(x0)
x = einops.rearrange(x0, "... (d j) -> ... d j", j=2)
print(x)
x1, x2 = x.unbind(dim=-1)

res = torch.stack((-x2, x1), dim=-1)
print(res)
einops.rearrange(res, "... d j -> ... (d j)")

tensor([0, 1, 2, 3, 4, 5])
tensor([[0, 1],
        [2, 3],
        [4, 5]])
tensor([[-1,  0],
        [-3,  2],
        [-5,  4]])


tensor([-1,  0, -3,  2, -5,  4])

## softmax

In [69]:
arr = torch.arange(27).reshape((3, 3, 3))
torch.max(arr, dim=1, keepdim=True)

torch.return_types.max(
values=tensor([[[ 6,  7,  8]],

        [[15, 16, 17]],

        [[24, 25, 26]]]),
indices=tensor([[[2, 2, 2]],

        [[2, 2, 2]],

        [[2, 2, 2]]]))

In [74]:
print(arr)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])


In [75]:
arr_max = torch.max(arr, dim=1, keepdim=True).values
print(arr_max)
print(arr_max.shape)
diff = arr - arr_max
print(diff)
print(diff.shape)

tensor([[[ 6,  7,  8]],

        [[15, 16, 17]],

        [[24, 25, 26]]])
torch.Size([3, 1, 3])
tensor([[[-6, -6, -6],
         [-3, -3, -3],
         [ 0,  0,  0]],

        [[-6, -6, -6],
         [-3, -3, -3],
         [ 0,  0,  0]],

        [[-6, -6, -6],
         [-3, -3, -3],
         [ 0,  0,  0]]])
torch.Size([3, 3, 3])


In [76]:
exp_res = torch.exp(diff)
print(exp_res)

tensor([[[0.0025, 0.0025, 0.0025],
         [0.0498, 0.0498, 0.0498],
         [1.0000, 1.0000, 1.0000]],

        [[0.0025, 0.0025, 0.0025],
         [0.0498, 0.0498, 0.0498],
         [1.0000, 1.0000, 1.0000]],

        [[0.0025, 0.0025, 0.0025],
         [0.0498, 0.0498, 0.0498],
         [1.0000, 1.0000, 1.0000]]])


In [78]:
sum_res = torch.sum(exp_res, dim=1, keepdim=True)
print(sum_res, sum_res.shape)

tensor([[[1.0523, 1.0523, 1.0523]],

        [[1.0523, 1.0523, 1.0523]],

        [[1.0523, 1.0523, 1.0523]]]) torch.Size([3, 1, 3])


In [80]:
softmax = exp_res / sum_res
print(softmax)
print(torch.sum(softmax, dim=1, keepdim=True))

tensor([[[0.0024, 0.0024, 0.0024],
         [0.0473, 0.0473, 0.0473],
         [0.9503, 0.9503, 0.9503]],

        [[0.0024, 0.0024, 0.0024],
         [0.0473, 0.0473, 0.0473],
         [0.9503, 0.9503, 0.9503]],

        [[0.0024, 0.0024, 0.0024],
         [0.0473, 0.0473, 0.0473],
         [0.9503, 0.9503, 0.9503]]])
tensor([[[1., 1., 1.]],

        [[1., 1., 1.]],

        [[1., 1., 1.]]])


## dim坍缩-内存布局

Note: dim=0 是最外层 size=4（如下所示）

In [86]:
x = torch.tensor([
    [[1, 2], [3, 4], [8, 9]],    # 层0
    [[5, 6], [7, 8], [1, 2]],     # 层1
    [[5, 6], [7, 8], [1, 2]],
    [[5, 6], [7, 8], [1, 2]]
]) 

x[0,:]

tensor([[1, 2],
        [3, 4],
        [8, 9]])

In [91]:
print(x.shape)
print(x[0])
x[0].shape

torch.Size([4, 3, 2])
tensor([[1, 2],
        [3, 4],
        [8, 9]])


torch.Size([3, 2])

In [87]:
print("dim=0 (压扁层):", torch.max(x, dim=0).values)

dim=0 (压扁层): tensor([[5, 6],
        [7, 8],
        [8, 9]])


## 掩码

- 【问题】Pytorch，一个矩阵、和掩码矩阵（true、false）。如何让true的位置不变，false的设置为某个值？



In [108]:
arr = torch.arange(4, dtype=torch.float32).reshape((2, 2))
masks = torch.tensor([[True, False], [True, False]], dtype=torch.bool)
print(masks)
print(torch.where(masks, arr, -100))
print(arr.masked_fill(masks, -100))
print(arr.masked_fill(~masks, -100))

tensor([[ True, False],
        [ True, False]])
tensor([[   0., -100.],
        [   2., -100.]])
tensor([[-100.,    1.],
        [-100.,    3.]])
tensor([[   0., -100.],
        [   2., -100.]])



补充说明
-   若需**原地修改**（节省内存）：
-   掩码形状需与原张量**可广播兼容**（如 `(3,4)` 张量可用 `(3,1)` 掩码）
-   所有方法均支持 GPU 张量，无需额外处理

In [109]:
print(arr)
arr[~masks] = torch.nan
print(arr)

tensor([[0., 1.],
        [2., 3.]])
tensor([[0., nan],
        [2., nan]])


In [111]:
print(arr)
arr[~masks] = torch.inf
print(arr)


tensor([[0., -inf],
        [2., -inf]])
tensor([[0., inf],
        [2., inf]])


## transpose

In [116]:
arr = torch.arange(16).reshape((2, 2, 2, 2))
print(arr)
print(arr[1, 1, :])

tensor([[[[ 0,  1],
          [ 2,  3]],

         [[ 4,  5],
          [ 6,  7]]],


        [[[ 8,  9],
          [10, 11]],

         [[12, 13],
          [14, 15]]]])
tensor([[12, 13],
        [14, 15]])


In [117]:
res = arr.transpose(-2, -1)
print(res)
print(res[1, 1, :])

tensor([[[[ 0,  2],
          [ 1,  3]],

         [[ 4,  6],
          [ 5,  7]]],


        [[[ 8, 10],
          [ 9, 11]],

         [[12, 14],
          [13, 15]]]])
tensor([[12, 14],
        [13, 15]])


In [119]:
arr = torch.arange(6).reshape((2, 3))
print(arr)
res = arr.transpose(-2, -1)
print(res)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


## 因果掩码

In [132]:
seq_len = 5
ones = torch.ones(seq_len, seq_len)
mask = torch.tril(ones)
print(mask)
mask.bool()

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])

In [133]:
mask.unsqueeze(0).unsqueeze(0)

tensor([[[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]]])

通过 “广播比较” 创建

In [126]:
positions = torch.arange(seq_len)
positions

tensor([0, 1, 2, 3, 4])

In [127]:
positions.unsqueeze(0)

tensor([[0, 1, 2, 3, 4]])

In [128]:
positions.unsqueeze(1)

tensor([[0],
        [1],
        [2],
        [3],
        [4]])

In [131]:
mask = positions.unsqueeze(0) <= positions.unsqueeze(1)
mask

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])